In [ ]:
import pandas as pd
import json

# 1. 첫 번째 파일(tbl_app_testhisdtl_202401.parquet) 불러오기
file_path = 'tbl_app_testhisdtl/tbl_app_testhisdtl_202401.parquet'  
df_tbl_app_testhisdtl = pd.read_parquet(file_path)

# 불필요한 열 삭제
df_tbl_app_testhisdtl.drop(columns=['No', 'Answer', 'Correct', 'CreDate', 'LM_IDX'], inplace=True)

# 2. 두 번째 파일(e_content_meta.csv) 불러오기
e_content_meta_file_path = 'e_content_meta.csv'
df_e_content_meta = pd.read_csv(e_content_meta_file_path)

# 데이터 필터링
df_2022 = df_e_content_meta[(df_e_content_meta['leccourse_nm'] == '2022 개정') & 
                            (df_e_content_meta['l_type_nm'] != '탭)한입 쏙 학교 공부 > 한입 문제')]

# 'l_type_nm'이 '기)교과서 진도학습'이고, 'l_title'이 '단원 마무리 문제'인 행을 제외
df_2022_filtered = df_2022[~((df_2022['l_type_nm'] == '기)교과서 진도학습') & 
                             (df_2022['l_title'] == '단원 마무리 문제'))]

# 불필요한 열 삭제
e_content_meta = df_2022_filtered.drop(columns=['unique_content_nm', 'l_type', 'service_1depth', 
                                                'service_2depth', 'subject', 'subject_nm', 'content_grade', 
                                                'content_grade_nm', 'term', 'term_nm', 'leccourse', 'leccourse_nm', 
                                                'l_level', 'l_info', 'l_template', 'l_template_nm', 'u_idx', 'u_parent_idx',
                                                'u_seq', 'u_type', 'u_lchapter', 'u_mchapter', 'textbooktype', 'u_title', 
                                                'l_idx', 'l_seq'])


# 3. 세 번째 파일(t_problem_analysis.csv) 불러오기
t_problem_analysis_file_path = 't_problem_analysis.csv'
df_t_problem_analysis = pd.read_csv(t_problem_analysis_file_path)

# 불필요한 열 삭제
t_problem_analysis = df_t_problem_analysis.drop(columns=['f_file_id', 'f_emh_cd', 'f_eduprocess_cd', 
                                                         'f_area_cd', 'f_area_cd', 'f_chapkind_cd', 
                                                         'f_schapter_id', 'f_tchapter_id', 'f_echapter_id', 
                                                         'f_difficult_cd', 'f_analysis_id', 'f_studytree_id', 
                                                         'f_test_cnt', 'f_good_cnt', 'f_deleteyn', 'f_mginfo', 
                                                         'f_subject_id'])

# 4. 데이터 병합
# 'mCode'와 'quizcode' 컬럼 이름 일치시키기
if 'mCode' in df_tbl_app_testhisdtl.columns:
    df_tbl_app_testhisdtl.rename(columns={'mCode': 'mcode'}, inplace=True)

if 'f_problem_id' in t_problem_analysis.columns:
    t_problem_analysis.rename(columns={'f_problem_id': 'quizcode'}, inplace=True)

if 'QuizCode' in df_tbl_app_testhisdtl.columns:
    df_tbl_app_testhisdtl.rename(columns={'QuizCode': 'quizcode'}, inplace=True)

# 'UserID'가 존재하면 'userid'로 컬럼명 변경
if 'UserID' in df_tbl_app_testhisdtl.columns:
    df_tbl_app_testhisdtl.rename(columns={'UserID': 'userid'}, inplace=True)

# 병합 전에 f_lchapter_id와 f_mchapter_id를 문자열로 변환
# 소수점을 제거하고 정수형으로 변환 후 문자열로 처리
t_problem_analysis['f_mchapter_id'] = t_problem_analysis['f_mchapter_id'].dropna().astype(int).astype(str)

# mcode를 기준으로 병합
merged_df = pd.merge(df_tbl_app_testhisdtl, e_content_meta, on='mcode', how='inner')

# 중복 행 제거
df_unique = merged_df.drop_duplicates()

# quizcode로 병합
merged_df1 = pd.merge(df_unique, t_problem_analysis, on='quizcode', how='inner')

# 최종 중복 제거
df_unique1 = merged_df1.drop_duplicates()

# 5. 널값 처리: 'f_mchapter_id', 'f_lchapter_id', 'userid' 컬럼의 널값 제거
df_unique1 = df_unique1.dropna(subset=['f_mchapter_id', 'f_lchapter_id', 'userid'])
df_unique1 = df_unique1.drop_duplicates(subset=['userid', 'f_lchapter_id', 'f_mchapter_id'])
# 6. 최종 데이터프레임에서 필요한 컬럼만 선택
df_final = df_unique1[['userid', 'mcode', 'l_title', 'leccode', 'f_lchapter_id', 'f_mchapter_id','l_type_nm']]
df_final = df_final.drop_duplicates(subset=['userid', 'mcode', 'f_mchapter_id'])
df_final.reset_index(drop=True, inplace=True)
# 7. 데이터프레임 정보 확인
print(df_final.info())  # 최종 데이터프레임 확인

# 8. 데이터프레임에서 'mcode', 'f_mchapter_id', 'userid' 컬럼 값 가져오기
mcodes = df_final['mcode'].unique()
f_mchapter_ids = df_final['f_mchapter_id'].unique()
user_ids = df_final['userid'].unique()

# 9. 타입별 독립적 ID 넘버링
# 학생 ID 넘버링
user_id_map = {user_id: idx for idx, user_id in enumerate(user_ids)}

# 중단원 ID 넘버링
f_mchapter_id_map = {f_mchapter_id: idx for idx, f_mchapter_id in enumerate(f_mchapter_ids)}

# 강의 ID 넘버링
mcode_map = {mcode: idx for idx, mcode in enumerate(mcodes)}

# 10. 타입별로 id_map 생성
id_map = {
    "students": user_id_map,
    "concepts": f_mchapter_id_map,
    "lectures": mcode_map
}

# id_map.json 저장
with open('id_map.json', 'w') as json_file:
    json.dump(id_map, json_file, indent=4)

# 결과 확인
print("id_map.json 파일이 생성되었습니다.")
df_final.to_csv("df_final.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232806 entries, 0 to 232805
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   userid         232806 non-null  object 
 1   mcode          232806 non-null  object 
 2   l_title        232806 non-null  object 
 3   leccode        232806 non-null  object 
 4   f_lchapter_id  232806 non-null  float64
 5   f_mchapter_id  232806 non-null  object 
 6   l_type_nm      232806 non-null  object 
dtypes: float64(1), object(6)
memory usage: 12.4+ MB
None
id_map.json 파일이 생성되었습니다.


139495

In [5]:
df_final

,userid,mcode,l_title,leccode,f_lchapter_id,f_mchapter_id,l_type_nm
0,f70467a4-7bc5-4bda-99d4-0280065daa65,T1ME11U01001,"1, 2, 3, 4, 5 알아보기",T1ME11U01,17122834.0,14201779,탭)겨울방학특강
1,04e62e1c-6062-4cb7-9042-027ee3cb02d9,T1ME11U01001,"1, 2, 3, 4, 5 알아보기",T1ME11U01,17122834.0,14201779,탭)겨울방학특강
2,18295854-6c88-4bdf-8613-024287b2adab,T1ME11U01001,"1, 2, 3, 4, 5 알아보기",T1ME11U01,17122834.0,14201779,탭)겨울방학특강
3,b775ea7e-eb6b-42c0-80ca-0317215ee3cc,T1ME11U01001,"1, 2, 3, 4, 5 알아보기",T1ME11U01,17122834.0,14201779,탭)겨울방학특강
4,4b1cf8ce-87b5-4058-8b9c-0308b23a08e7,T1ME11U01001,"1, 2, 3, 4, 5 알아보기",T1ME11U01,17122834.0,14201779,탭)겨울방학특강
...,...,...,...,...,...,...,...
232801,f35fec7a-a778-4377-9e66-ca9ffe4e4367,T1ME11U02008,모양 찾기 놀이를 해 볼까요,T1ME11U02,17122835.0,14201784,기)교과서 진도학습
232802,f35fec7a-a778-4377-9e66-ca9ffe4e4367,T1ME11U02008,모양 찾기 놀이를 해 볼까요,T1ME11U02,17122835.0,14201785,기)교과서 진도학습
232803,89df8b3c-91c8-420f-ad7d-adb4b72bf00e,T1ME11U01064,1. 9까지의 수,T1ME11U01,17122834.0,14201779,탭)AI 추천 문제
232804,89df8b3c-91c8-420f-ad7d-adb4b72bf00e,T1ME11U01064,1. 9까지의 수,T1ME11U01,17122834.0,14201780,탭)AI 추천 문제


In [5]:
import pandas as pd
import json

# 1. 첫 번째 파일(tbl_app_testhisdtl_202401.parquet) 불러오기
file_path = 'tbl_app_testhisdtl/tbl_app_testhisdtl_202401.parquet'  
df_tbl_app_testhisdtl = pd.read_parquet(file_path)

# 불필요한 열 삭제
df_tbl_app_testhisdtl.drop(columns=['No', 'Answer', 'Correct', 'CreDate', 'LM_IDX'], inplace=True)

# 2. 두 번째 파일(e_content_meta.csv) 불러오기
e_content_meta_file_path = 'e_content_meta.csv'
df_e_content_meta = pd.read_csv(e_content_meta_file_path)

# 데이터 필터링
df_2022 = df_e_content_meta[(df_e_content_meta['leccourse_nm'] == '2022 개정') & 
                            (df_e_content_meta['l_type_nm'] != '탭)한입 쏙 학교 공부 > 한입 문제')]

# 'l_type_nm'이 '기)교과서 진도학습'이고, 'l_title'이 '단원 마무리 문제'인 행을 제외
df_2022_filtered = df_2022[~((df_2022['l_type_nm'] == '기)교과서 진도학습') & 
                             (df_2022['l_title'] == '단원 마무리 문제'))]

# 불필요한 열 삭제
e_content_meta = df_2022_filtered.drop(columns=['unique_content_nm', 'l_type', 'service_1depth', 
                                                'service_2depth', 'subject', 'subject_nm', 'content_grade', 
                                                'content_grade_nm', 'term', 'term_nm', 'leccourse', 'leccourse_nm', 
                                                'l_level', 'l_info', 'l_template', 'l_template_nm', 'u_idx', 'u_parent_idx',
                                                'u_seq', 'u_type', 'u_lchapter', 'u_mchapter', 'textbooktype', 'u_title', 
                                                'l_idx', 'l_seq'])


# 3. 세 번째 파일(t_problem_analysis.csv) 불러오기
t_problem_analysis_file_path = 't_problem_analysis.csv'
df_t_problem_analysis = pd.read_csv(t_problem_analysis_file_path)

# 불필요한 열 삭제
t_problem_analysis = df_t_problem_analysis.drop(columns=['f_file_id', 'f_emh_cd', 'f_eduprocess_cd', 
                                                         'f_area_cd', 'f_area_cd', 'f_chapkind_cd', 
                                                         'f_schapter_id', 'f_tchapter_id', 'f_echapter_id', 
                                                         'f_difficult_cd', 'f_analysis_id', 'f_studytree_id', 
                                                         'f_test_cnt', 'f_good_cnt', 'f_deleteyn', 'f_mginfo', 
                                                         'f_subject_id'])

# 4. 데이터 병합
# 'mCode'와 'quizcode' 컬럼 이름 일치시키기
if 'mCode' in df_tbl_app_testhisdtl.columns:
    df_tbl_app_testhisdtl.rename(columns={'mCode': 'mcode'}, inplace=True)

if 'f_problem_id' in t_problem_analysis.columns:
    t_problem_analysis.rename(columns={'f_problem_id': 'quizcode'}, inplace=True)

if 'QuizCode' in df_tbl_app_testhisdtl.columns:
    df_tbl_app_testhisdtl.rename(columns={'QuizCode': 'quizcode'}, inplace=True)

# 'UserID'가 존재하면 'userid'로 컬럼명 변경
if 'UserID' in df_tbl_app_testhisdtl.columns:
    df_tbl_app_testhisdtl.rename(columns={'UserID': 'userid'}, inplace=True)

# 병합 전에 f_lchapter_id와 f_mchapter_id를 문자열로 변환
# 소수점을 제거하고 정수형으로 변환 후 문자열로 처리
t_problem_analysis['f_mchapter_id'] = t_problem_analysis['f_mchapter_id'].dropna().astype(int).astype(str)

# mcode를 기준으로 병합
merged_df = pd.merge(df_tbl_app_testhisdtl, e_content_meta, on='mcode', how='inner')

# 중복 행 제거
df_unique = merged_df.drop_duplicates()

# quizcode로 병합
merged_df1 = pd.merge(df_unique, t_problem_analysis, on='quizcode', how='inner')

# 최종 중복 제거
df_unique1 = merged_df1.drop_duplicates()

# 5. 널값 처리: 'f_mchapter_id', 'f_lchapter_id', 'userid' 컬럼의 널값 제거
df_unique1 = df_unique1.dropna(subset=['f_mchapter_id', 'f_lchapter_id', 'userid'])
df_unique1 = df_unique1.drop_duplicates(subset=['userid', 'f_lchapter_id', 'f_mchapter_id'])
# 6. 최종 데이터프레임에서 필요한 컬럼만 선택
df_final = df_unique1[['userid', 'mcode', 'l_title', 'leccode', 'f_lchapter_id', 'f_mchapter_id','l_type_nm']]
df_final = df_final.drop_duplicates(subset=['userid', 'mcode', 'f_mchapter_id'])
df_final.reset_index(drop=True, inplace=True)
# 7. 데이터프레임 정보 확인
print(df_final.info())  # 최종 데이터프레임 확인

# 8. 데이터프레임에서 'mcode', 'f_mchapter_id', 'userid' 컬럼 값 가져오기
mcodes = df_final['mcode'].unique()
f_mchapter_ids = df_final['f_mchapter_id'].unique()
user_ids = df_final['userid'].unique()

# 9. 넘버링: 각 고유한 값들에 대해 인덱스를 매기기
# userid 넘버링은 0번부터 시작
start_user = 0

# userid 넘버링 (0번부터 시작)
user_id_map = {user_id: idx + start_user for idx, user_id in enumerate(user_ids)}

# f_mchapter_id 넘버링 (userid 끝번호 이후부터 시작)
start_fmchapter = start_user + len(user_ids)
f_mchapter_id_map = {f_mchapter_id: idx + start_fmchapter for idx, f_mchapter_id in enumerate(f_mchapter_ids)}

# mcode 넘버링 (f_mchapter_id 끝번호 이후부터 시작)
start_mcode = start_fmchapter + len(f_mchapter_ids)
mcode_map = {mcode: idx + start_mcode for idx, mcode in enumerate(mcodes)}

# 10. 최종 id_map 생성
id_map = {}
id_map.update(user_id_map)
id_map.update(f_mchapter_id_map)
id_map.update(mcode_map)

# id_map의 모든 값을 정수로 유지
id_map = {key: int(value) for key, value in id_map.items()}

# 11. json 파일로 저장
with open('id_map.json', 'w') as json_file:
    json.dump(id_map, json_file, indent=4)

# 결과 확인
print("id_map.json 파일이 생성되었습니다.")
df_final.to_csv("df_final.txt", index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232806 entries, 0 to 232805
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   userid         232806 non-null  object 
 1   mcode          232806 non-null  object 
 2   l_title        232806 non-null  object 
 3   leccode        232806 non-null  object 
 4   f_lchapter_id  232806 non-null  float64
 5   f_mchapter_id  232806 non-null  object 
 6   l_type_nm      232806 non-null  object 
dtypes: float64(1), object(6)
memory usage: 12.4+ MB
None
id_map.json 파일이 생성되었습니다.


In [51]:
# 그래프의 id 생성
import pandas as pd
import json

# 데이터프레임에서 unique 값 추출
user_ids = df_final['userid'].unique()
f_mchapter_ids = df_final['f_mchapter_id'].unique()
mcodes = df_final['mcode'].unique()  # f_lchapter_id 대신 mcode를 가져옴

# 노드 정보 생성
nodes = []

# 'user' 타입 노드 추가
for user_id in user_ids:
    nodes.append({"id": user_id, "type": "user"})

# 'mchapter' 타입 노드 추가
for mchapter_id in f_mchapter_ids:
    nodes.append({"id": mchapter_id, "type": "mchapter"})

# 'mcode' 타입 노드 추가 (f_lchapter_id 대신 mcode를 사용)
for mcode in mcodes:
    nodes.append({"id": mcode, "type": "mcode"})  # 'lchapter' 대신 'mcode' 타입으로 추가

# JSON 구조 생성
node_data = {"nodes": nodes}

# JSON 파일로 저장
with open("node_id.json", "w") as json_file:
    json.dump(node_data, json_file, indent=4)

print("node_id.json 파일이 생성되었습니다.")


node_id.json 파일이 생성되었습니다.


In [49]:
import random
import json

# df_final이 준비된 상태여야 합니다.

# 1. unique 값 추출
unique_user_ids = df_final['userid'].unique()
unique_f_mchapter_ids = df_final['f_mchapter_id'].unique()
unique_mcode_ids = df_final['mcode'].unique()  # f_lchapter_id -> mcode로 변경

# 2. "userid -> f_mchapter_id" 엣지 생성 (유니크한 값 기준)
user_to_mchapter_edges = []

for _, row in df_final.drop_duplicates(subset=['userid', 'f_mchapter_id']).iterrows():
    weight = round(random.uniform(0, 1), 2)  # 랜덤 가중치 (0~1.0)
    user_to_mchapter_edges.append({
        "source": row["userid"],
        "target": row["f_mchapter_id"],
        "weight": weight
    })

# 3. "f_mchapter_id -> mcode" 엣지 생성 (유니크한 값 기준)
mchapter_to_mcode_edges = []

for _, row in df_final.drop_duplicates(subset=['f_mchapter_id', 'mcode']).iterrows():
    mchapter_to_mcode_edges.append({
        "source": row["f_mchapter_id"],
        "target": row["mcode"],  # 'f_lchapter_id'를 'mcode'로 변경
    })

# 4. 모든 엣지 합치기
all_edges = user_to_mchapter_edges + mchapter_to_mcode_edges

# 5. JSON 구조 생성
graph = {"edges": all_edges}  # "links" -> "edges"

# 6. JSON 파일 저장
with open('edge.json', 'w') as json_file:
    json.dump(graph, json_file, indent=4)

print("edge.json 파일이 성공적으로 생성되었습니다.")



edge.json 파일이 성공적으로 생성되었습니다.


In [45]:
import random
import json

# df_final이 준비된 상태여야 합니다.

# 1. unique 값 추출
unique_user_ids = df_final['userid'].unique()
unique_f_mchapter_ids = df_final['f_mchapter_id'].unique()
unique_mcodes = df_final['mcode'].unique()

# 2. "userid -> f_mchapter_id" 엣지 생성 (유니크한 값 기준)
user_to_mchapter_edges = []  # links -> edges로 변경

for _, row in df_final.drop_duplicates(subset=['userid', 'f_mchapter_id']).iterrows():
    weight = round(random.uniform(0, 1), 2)  # 랜덤 가중치 (0~1.0)
    user_to_mchapter_edges.append({
        "source": row["userid"],
        "target": row["f_mchapter_id"],
        "weight": weight
    })

# 3. "f_mchapter_id -> mcode" 엣지 생성 (유니크한 값 기준)
mchapter_to_mcode_edges = []  # links -> edges로 변경

for _, row in df_final.drop_duplicates(subset=['f_mchapter_id', 'mcode']).iterrows():
    mchapter_to_mcode_edges.append({
        "source": row["f_mchapter_id"],
        "target": row["mcode"]
    })

# 4. f_mchapter_id 값들끼리의 연결 추가
f_mchapter_edges = []

# f_mchapter_id 값들끼리의 모든 조합 생성
for i, f_mchapter_id_1 in enumerate(unique_f_mchapter_ids):
    for j, f_mchapter_id_2 in enumerate(unique_f_mchapter_ids):
        if i != j:  # 동일한 값은 제외 (자기 자신과의 연결은 제외)
            f_mchapter_edges.append({
                "source": f_mchapter_id_1,
                "target": f_mchapter_id_2
            })

# 5. 모든 엣지 합치기
all_edges = user_to_mchapter_edges + mchapter_to_mcode_edges + f_mchapter_edges

# 중복된 엣지 제거: 이 부분을 다시 점검하여 단순한 중복 제거 방식으로 처리
seen_edges = set()
unique_edges = []

for edge in all_edges:
    # 엣지의 source와 target을 튜플로 하여 set에 저장해 중복을 체크
    edge_tuple = (edge["source"], edge["target"])
    if edge_tuple not in seen_edges:
        seen_edges.add(edge_tuple)
        unique_edges.append(edge)

# 6. JSON 구조 생성
graph = {"edges": unique_edges}

# 7. JSON 파일 저장
with open('edge.json', 'w') as json_file:
    json.dump(graph, json_file, indent=4)

print("edge.json 파일이 성공적으로 생성되었습니다.")



edge.json 파일이 성공적으로 생성되었습니다.


In [52]:
# graph
import json

node_file_path = "node_id.json"
link_file_path = "edge.json"

with open(node_file_path, "r") as node_file:
    node_data = json.load(node_file)

with open(link_file_path, "r") as link_file:
    link_data = json.load(link_file)

# 'id' 기준으로 중복 제거
unique_nodes = {node["id"]: node for node in node_data["nodes"]}.values()

# 2. 중복 제거: 링크
# ('source', 'target') 기준으로 중복 제거
unique_edges = {(link["source"], link["target"]): link for link in link_data["edges"]}.values()

# 3. G.json 생성
graph = {
    # "directed": False,
    # "graph": {},
    "nodes": list(unique_nodes),  # 중복 제거된 노드 정보
    "edges": list(unique_edges)   # 중복 제거된 링크 정보
}

# 4. G.json 파일 저장
with open("G.json", "w") as json_file:
    json.dump(graph, json_file, indent=4)

print("G.json 파일이 성공적으로 생성되었습니다. 모든 중복이 제거되었습니다.")


G.json 파일이 성공적으로 생성되었습니다. 모든 중복이 제거되었습니다.


In [76]:
import random
import pandas as pd
import json

# CSV 파일 읽기
df_final = pd.read_csv("df_final.csv")

# 1. "userid -> f_mchapter_id" 엣지 생성 (유니크한 값 기준)
user_to_mchapter_data = []

# `userid`와 `f_mchapter_id`를 이용하여 데이터 생성
for _, row in df_final.drop_duplicates(subset=['userid', 'f_mchapter_id']).iterrows():
    user_to_mchapter_data.append([row["userid"], str(row["f_mchapter_id"])])

# 2. "f_mchapter_id -> mcode" 엣지 생성 (유니크한 값 기준)
mchapter_to_mcode_data = []

for _, row in df_final.drop_duplicates(subset=['f_mchapter_id', 'mcode']).iterrows():
    mchapter_to_mcode_data.append([str(row["f_mchapter_id"]), str(row["mcode"])])

# 3. 두 개의 JSON 파일로 저장

# 2번 "userid -> f_mchapter_id" 엣지 데이터 JSON 저장
user_to_mchapter_json = 'user_to_mchapter_data.json'
with open(user_to_mchapter_json, mode='w') as file:
    json.dump(user_to_mchapter_data, file, indent=4)

# 3번 "f_mchapter_id -> mcode" 엣지 데이터 JSON 저장
mchapter_to_mcode_json = 'mchapter_to_mcode_data.json'
with open(mchapter_to_mcode_json, mode='w') as file:
    json.dump(mchapter_to_mcode_data, file, indent=4)

print(f"{user_to_mchapter_json}와 {mchapter_to_mcode_json} 파일이 성공적으로 생성되었습니다.")



user_to_mchapter_data.json와 mchapter_to_mcode_data.json 파일이 성공적으로 생성되었습니다.
